In [2]:
%load_ext autoreload
%autoreload 2

import re
import os
import json
import time
import sqlite3
import requests
import datetime
import numpy as np
from tqdm.notebook import tqdm, trange
from multiprocessing import Manager, Pool

# from src import DataBase

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
class DataBase:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.database = None
        self.cursor = None
        

    def initialize(self, create: bool = False):
        self.database = sqlite3.connect(self.db_path, check_same_thread=False)
        self.cursor = self.database.cursor()
  
        if create:
            _cmd = f"CREATE TABLE submissions " \
                   f"(id int, url text, title text, keywords text, " \
                   f"rating_0_cnt int, rating_0_avg float, rating_0_std float, ratings_0 text," \
                   f"deision text, withdraw int )"
            self.cursor.execute(_cmd)
    
    def write_item_rating(
        self, 
        suf: int,
        _id: int, 
        ratings: list,
        withdraw: int,
    ):
#         title = title.replace('\\', '').replace("\"", "'")
        num_rating = len(ratings)
        rating_avg = np.mean(ratings).item()
        rating_std = np.std(ratings).item()
        ratings = ', '.join(map(str, ratings))
        _cmd = f"UPDATE submissions " \
               f"SET " \
               f"rating_{suf}_cnt = {num_rating}, rating_{suf}_avg = {rating_avg}, " \
               f"rating_{suf}_std = {rating_std}, " \
               f"ratings_{suf} = \"{ratings}\", withdraw = {withdraw} " \
               f"WHERE id = {_id}"
#         print(_cmd)
        self.cursor.execute(_cmd)
        self.database.commit()
        

    def close(self):
        self.cursor.close()
        self.database.close()

In [4]:
db = DataBase('../assets/_iclr2022.db')
db.initialize(create=False)

In [12]:
db.cursor.execute("ALTER TABLE submissions ADD COLUMN rating_3_cnt int")
db.cursor.execute("ALTER TABLE submissions ADD COLUMN rating_3_avg float")
db.cursor.execute("ALTER TABLE submissions ADD COLUMN rating_3_std float")
db.cursor.execute("ALTER TABLE submissions ADD COLUMN ratings_3 text")

In [5]:
db.cursor.execute('SELECT id, url FROM submissions')
data = db.cursor.fetchall()
db.close()

## Parse each item

In [6]:
def parse_item(i, _id, url, db_path):
#     print(f"-->>: {i}: {url}")
    
    item_id = url.split('id=')[-1]
    rqst_url = f"https://api.openreview.net/notes?forum={item_id}&" \
          f"trash=true&details=replyCount%2Cwritable%2Crevisions%2Coriginal%2Coverwriting%2Cinvitation%2Ctags"
    rqst_data = requests.get(rqst_url)
    rqst_data = json.loads(rqst_data.text)
    
    # parse each note
    withdraw = 0
    # filter meta note
    meta_note = [d for d in rqst_data['notes'] if 'Paper' not in d['invitation']]
    # check withdrawn
    withdraw = 1 if 'Withdrawn_Submission' in meta_note[0]['invitation'] else 0
    # filter reviewer comments
    comment_notes = [d for d in rqst_data['notes'] \
                     if 'Official_Review' in d['invitation'] and 'recommendation' in d['content'].keys()]
    comment_notes = sorted(comment_notes, key=lambda d: d['number'])[::-1]
    ratings = [int(note['content']['recommendation'].split(':')[0]) for note in comment_notes]
    
    db = DataBase(db_path)
    db.initialize(create=False)
    db.write_item_rating(3, _id, ratings, withdraw)
    print(f"done: {i}")
    db.close()

    
p = Pool(processes=5)
num_items = 20  # len(data)
print(f"Processing {num_items} items:\n")
for i in range(num_items):
    _id = data[i][0]
    url = data[i][1].strip()
    p.apply_async(parse_item, args=(i, _id, url, '../assets/_iclr2022.db'))
p.close()
p.join()

Processing 20 items:

done: 4
done: 2
done: 1
done: 3
done: 0
done: 9
done: 7
done: 6
done: 8
done: 5
done: 10
done: 11
done: 12
done: 13
done: 14
done: 15
done: 18
done: 17
done: 16
done: 19
